<a href="https://colab.research.google.com/github/gautampatil01/Recommendor-system/blob/main/Recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
#https://api.themoviedb.org/3/movie/168259?api_key=62e6915f9db9a03352dc663b7e1449f4

In [162]:
import pandas as pd
import numpy as np
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [97]:
!git clone https://github.com/gautampatil01/Recommendor-system

fatal: destination path 'Recommendor-system' already exists and is not an empty directory.


In [98]:
!ls

drive  Recommendor-system  sample_data	tmdb_movies_data.csv


In [99]:
netflix  =pd.read_csv('/content/Recommendor-system/tmdb_movies_data.csv')
df = netflix.copy()

In [100]:
df.head(2)

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,137999939.3,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,137999939.3,3.481613e+08


In [101]:
df.columns

Index(['id', 'imdb_id', 'popularity', 'budget', 'revenue', 'original_title',
       'cast', 'homepage', 'director', 'tagline', 'keywords', 'overview',
       'runtime', 'genres', 'production_companies', 'release_date',
       'vote_count', 'vote_average', 'release_year', 'budget_adj',
       'revenue_adj'],
      dtype='object')

In [102]:
df = df[['id', 'original_title', 'cast', 'director', 'keywords', 'overview', 'genres']]

In [103]:
df.head(2)

,id,original_title,cast,director,keywords,overview,genres
0,135397,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,monster|dna|tyrannosaurus rex|velociraptor|island,Twenty-two years after the events of Jurassic ...,Action|Adventure|Science Fiction|Thriller
1,76341,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,future|chase|post-apocalyptic|dystopia|australia,An apocalyptic story set in the furthest reach...,Action|Adventure|Science Fiction|Thriller


In [104]:
df.shape

(10866, 7)

In [105]:
df.isnull().sum()

id                   0
original_title       0
cast                76
director            44
keywords          1493
overview             4
genres              23
dtype: int64

In [106]:
df.dropna(inplace = True)

In [107]:
def splitter(column):
  df[column] = df[column].apply(lambda x: x.split('|'))

In [108]:
splitter('cast')
df.cast

0        [Chris Pratt, Bryce Dallas Howard, Irrfan Khan...
1        [Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...
2        [Shailene Woodley, Theo James, Kate Winslet, A...
3        [Harrison Ford, Mark Hamill, Carrie Fisher, Ad...
4        [Vin Diesel, Paul Walker, Jason Statham, Miche...
                               ...                        
10861    [Michael Hynson, Robert August, Lord 'Tally Ho...
10862    [James Garner, Eva Marie Saint, Yves Montand, ...
10863    [Innokentiy Smoktunovskiy, Oleg Efremov, Georg...
10864    [Tatsuya Mihashi, Akiko Wakabayashi, Mie Hama,...
10865    [Harold P. Warren, Tom Neyman, John Reynolds, ...
Name: cast, Length: 9306, dtype: object

In [109]:
splitter('director')
df.director

0           [Colin Trevorrow]
1             [George Miller]
2          [Robert Schwentke]
3               [J.J. Abrams]
4                 [James Wan]
                 ...         
10861           [Bruce Brown]
10862    [John Frankenheimer]
10863        [Eldar Ryazanov]
10864           [Woody Allen]
10865      [Harold P. Warren]
Name: director, Length: 9306, dtype: object

In [110]:
splitter('keywords')
df.keywords

0        [monster, dna, tyrannosaurus rex, velociraptor...
1        [future, chase, post-apocalyptic, dystopia, au...
2        [based on novel, revolution, dystopia, sequel,...
3              [android, spaceship, jedi, space opera, 3d]
4                [car race, speed, revenge, suspense, car]
                               ...                        
10861                         [surfer, surfboard, surfing]
10862                        [car race, racing, formula 1]
10863                         [car, trolley, stealing car]
10864                                              [spoof]
10865            [fire, gun, drive, sacrifice, flashlight]
Name: keywords, Length: 9306, dtype: object

In [111]:
splitter('genres')
df.genres

0        [Action, Adventure, Science Fiction, Thriller]
1        [Action, Adventure, Science Fiction, Thriller]
2                [Adventure, Science Fiction, Thriller]
3         [Action, Adventure, Science Fiction, Fantasy]
4                             [Action, Crime, Thriller]
                              ...                      
10861                                     [Documentary]
10862                        [Action, Adventure, Drama]
10863                                 [Mystery, Comedy]
10864                                  [Action, Comedy]
10865                                          [Horror]
Name: genres, Length: 9306, dtype: object

In [112]:
df['overview'] = df['overview'].apply(lambda x: x.split(' '))

In [118]:
df.overview[0]

['Twenty-two',
 'years',
 'after',
 'the',
 'events',
 'of',
 'Jurassic',
 'Park,',
 'Isla',
 'Nublar',
 'now',
 'features',
 'a',
 'fully',
 'functioning',
 'dinosaur',
 'theme',
 'park,',
 'Jurassic',
 'World,',
 'as',
 'originally',
 'envisioned',
 'by',
 'John',
 'Hammond.']

In [119]:
df['cast'] = df['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
df['keywords'] = df['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
df['genres'] = df['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
df['director'] = df['director'].apply(lambda x: [i.replace(" ", "") for i in x])

In [115]:
df.cast

0        [ChrisPratt, BryceDallasHoward, IrrfanKhan, Vi...
1        [TomHardy, CharlizeTheron, HughKeays-Byrne, Ni...
2        [ShaileneWoodley, TheoJames, KateWinslet, Anse...
3        [HarrisonFord, MarkHamill, CarrieFisher, AdamD...
4        [VinDiesel, PaulWalker, JasonStatham, Michelle...
                               ...                        
10861    [MichaelHynson, RobertAugust, Lord'TallyHo'Ble...
10862    [JamesGarner, EvaMarieSaint, YvesMontand, Tosh...
10863    [InnokentiySmoktunovskiy, OlegEfremov, GeorgiZ...
10864    [TatsuyaMihashi, AkikoWakabayashi, MieHama, Jo...
10865    [HaroldP.Warren, TomNeyman, JohnReynolds, Dian...
Name: cast, Length: 9306, dtype: object

In [121]:
df.head(1)

,id,original_title,cast,director,keywords,overview,genres
0,135397,Jurassic World,"[ChrisPratt, BryceDallasHoward, IrrfanKhan, Vi...",[ColinTrevorrow],"[monster, dna, tyrannosaurusrex, velociraptor,...","[Twenty-two, years, after, the, events, of, Ju...","[Action, Adventure, ScienceFiction, Thriller]"


In [122]:
df['tags'] = df['cast'] + df['director'] + df['keywords'] + df['overview'] + df['genres']

In [124]:
df.tags[0]

['ChrisPratt',
 'BryceDallasHoward',
 'IrrfanKhan',
 "VincentD'Onofrio",
 'NickRobinson',
 'ColinTrevorrow',
 'monster',
 'dna',
 'tyrannosaurusrex',
 'velociraptor',
 'island',
 'Twenty-two',
 'years',
 'after',
 'the',
 'events',
 'of',
 'Jurassic',
 'Park,',
 'Isla',
 'Nublar',
 'now',
 'features',
 'a',
 'fully',
 'functioning',
 'dinosaur',
 'theme',
 'park,',
 'Jurassic',
 'World,',
 'as',
 'originally',
 'envisioned',
 'by',
 'John',
 'Hammond.',
 'Action',
 'Adventure',
 'ScienceFiction',
 'Thriller']

In [126]:
df['tags'] = df['tags'].apply(lambda x: ' '.join(x))

In [128]:
df['tags'][0]

"ChrisPratt BryceDallasHoward IrrfanKhan VincentD'Onofrio NickRobinson ColinTrevorrow monster dna tyrannosaurusrex velociraptor island Twenty-two years after the events of Jurassic Park, Isla Nublar now features a fully functioning dinosaur theme park, Jurassic World, as originally envisioned by John Hammond. Action Adventure ScienceFiction Thriller"

In [148]:
ps = PorterStemmer()

In [149]:
def stemming(text):
  li = []
  for i in text.split():
    li.append(ps.stem(i))
  return ' '.join(li)

In [150]:
df['tags'] = df['tags'].apply(stemming)

In [152]:
df['tags'][0]

"chrispratt brycedallashoward irrfankhan vincentd'onofrio nickrobinson colintrevorrow monster dna tyrannosaurusrex velociraptor island twenty-two year after the event of jurass park, isla nublar now featur a fulli function dinosaur theme park, jurass world, as origin envis by john hammond. action adventur sciencefict thriller"

In [157]:
cv = CountVectorizer()

In [158]:
vectors = cv.fit_transform(df['tags']).toarray()

In [160]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [161]:
cv.get_feature_names_out().tolist()

['00',
 '000',
 '000km',
 '000th',
 '007',
 '01',
 '05pm',
 '07am',
 '10',
 '100',
 '1000',
 '1001',
 '100th',
 '101',
 '103',
 '108',
 '10th',
 '11',
 '1138',
 '114',
 '117',
 '1183',
 '119',
 '11th',
 '12',
 '1200',
 '1215',
 '125',
 '1250',
 '125th',
 '127',
 '12th',
 '12thcenturi',
 '13',
 '130',
 '1300',
 '13th',
 '14',
 '140',
 '1408',
 '142',
 '1429',
 '1475',
 '148',
 '1492',
 '14enn',
 '14pm',
 '14th',
 '15',
 '150',
 '150000',
 '150th',
 '1536',
 '1562',
 '1564',
 '15th',
 '15thcenturi',
 '15yr',
 '16',
 '1600',
 '1600s',
 '161',
 '1610',
 '1630s',
 '165',
 '1671',
 '168',
 '1681',
 '1691',
 '1692',
 '16mm',
 '16th',
 '16thcenturi',
 '17',
 '1700',
 '1700s',
 '174',
 '1787',
 '17th',
 '17thcenturi',
 '18',
 '1800',
 '1800s',
 '1812',
 '1814',
 '1816',
 '1818',
 '1820',
 '1820s',
 '1824',
 '1825',
 '183',
 '1830',
 '1836',
 '1839',
 '1840',
 '1845',
 '1846',
 '1850',
 '1851',
 '1857',
 '1860',
 '1863',
 '1865',
 '1868',
 '187',
 '1873',
 '1875',
 '1879',
 '1880',
 '1880s',
 '1